# Sequence-Structure Coverage

In [1]:
from pyspark.sql import SparkSession
from mmtfPyspark.structureViewer import view_binding_site
from mmtfPyspark.io import mmtfReader
from pyspark.sql.functions import concat_ws
from mmtfPyspark.datasets import g2sDataset, pdbjMineDataset, myVariantDataset

#### Configure Spark

In [2]:
spark = SparkSession.builder.master("local[4]").appName("sequence_structure_coverage").getOrCreate()
sc = spark.sparkContext

In [3]:
path = "/Users/kyle_stiers/Desktop/mmtf_files/reduced/"
pdb = mmtfReader.read_sequence_file(path, sc)

uniprotQuery = "SELECT * FROM sifts.pdb_chain_uniprot"
uniprot = pdbjMineDataset.get_dataset(uniprotQuery)

uniprot.toPandas().head(5)

,pdbid,chain,sp_primary,res_beg,res_end,pdb_beg,pdb_end,sp_beg,sp_end,structureChainId
0,101M,A,P02185,1,154,0,153,1,154,101M.A
1,102L,A,P00720,1,40,1,40,1,40,102L.A
2,102L,A,P00720,42,165,41,None,41,164,102L.A
3,102M,A,P02185,1,154,0,153,1,154,102M.A
4,103L,A,P00720,1,40,1,None,1,40,103L.A


In [ ]:
spark.stop()